## Imports

In [4]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## LSUV

In [6]:
#export
from exp.nb_07 import *

In [7]:
x_train,y_train,x_valid,y_valid = get_data()

x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [8]:
mnist_view = view_tfm(1,28,28)
cbfs = [Recorder,
        partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, mnist_view)]

In [9]:
nfs = [8,16,32,64,64]
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
    
    def forward(self, x): return self.relu(self.conv(x))
    
#     @property
#     def bias(self): return -self.relu.sub
#     @bias.setter
#     def bias(self,v): self.relu.sub = -v
#     @property
#     def weight(self): return self.conv.weight

In [10]:
learn,run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

In [11]:
Learner.children = lambda x: list(x.model.children())

In [16]:
learn.model

Sequential(
  (0): ConvLayer(
    (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (relu): GeneralRelu()
  )
  (1): ConvLayer(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (2): ConvLayer(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (3): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (4): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (5): AdaptiveAvgPool2d(output_size=1)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

In [76]:
def lin_chec(m, cond):
    if cond(m):
#         print(m)
        return [m]
    else:
#         print('first',m)
#         print('sum', sum([lin_chec(c, cond) for c in m.children()], [2]))
#         return [3]
        return sum([lin_chec(c, cond) for c in m.children()], [])

In [81]:
lin_chec(nn.Conv2d(3, 16, 5), lambda o : isinstance(o, nn.Conv2d))

[Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))]

In [79]:
lin_chec(learn.model, lambda o : isinstance(o, nn.Conv2d))

[Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
 Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))]

In [82]:
t=sum([lin_chec(learn.model, lambda o : isinstance(o, nn.Conv2d))], [1])

In [83]:
t

[1,
 Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)),
 Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))]

In [88]:
learn.model

Sequential(
  (0): ConvLayer(
    (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (relu): GeneralRelu()
  )
  (1): ConvLayer(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (2): ConvLayer(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (3): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (4): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu): GeneralRelu()
  )
  (5): AdaptiveAvgPool2d(output_size=1)
  (6): Lambda()
  (7): Linear(in_features=64, out_features=10, bias=True)
)

In [112]:
def find(m):
    if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Conv3d)): 
        print(m)
        return [m]
    for c in m.children():
#         print(c)
        find(c)
    return 4

In [113]:
find(learn.model)

Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))


4

### Recursive exps

In [147]:
import time
t=time.time()

In [160]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fibonacci_recursive(n):
    print("Calculating F", "(", n, ")", sep="", end=", ")
#     t1=time.time()
#     print(f'Time@{n}@{t1}')

    # Base case
    if n == 0:
        return 0
    elif n == 1:
        return 1

    # Recursive case
    else:
        return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)

In [162]:
fibonacci_recursive(5)

5

In [232]:
def rec_test(x):
    if x>0:
        print(x)
        t = [x] if (x%2==0) else []
        rec_test(x-1)
        return t
    else:
        return []

In [236]:
rec_test(10)

10
9
8
7
6
5
4
3
2
1


[10]

In [229]:
def rec_test(x):
    if x>0:
        print(x)
        t = [x] if (x%2==0) else [] 
        return sum([rec_test(x-1)], t)
    else:
        return []

In [231]:
rec_test(10)

10
9
8
7
6
5
4
3
2
1


[10, 8, 6, 4, 2]